In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn import feature_extraction, linear_model, model_selection, metrics
from sklearn import ensemble
from scipy import sparse
import sys
sys.path.append("..")
from src.make_model_lstm import hate_speech_model
from src.lstm_cleaner import lstm_cleaner

/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
#watch overfitting


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
df = pd.read_csv('../data/labeled_data.csv')

In [5]:
tweets = df['tweet'].apply(lambda x: x.lstrip('!'))

In [6]:
y = np.argmax(df[['hate_speech', 'offensive_language', 'neither']].values, axis = 1)

In [7]:
stop_words = set(stopwords.words('english')) 

In [8]:
token = nltk.tokenize.casual.TweetTokenizer()

In [10]:
lemmatizer = WordNetLemmatizer()

In [11]:
cleaned = []
wordDict = {}
i = 1
for tweet in tweets:
    tokenized = token.tokenize(tweet)
    newSent = []
    for word in tokenized:
        if word not in stop_words:
            newWord = lemmatizer.lemmatize(word)
            if newWord not in wordDict:
                wordDict[newWord] = i
                i += 1
            newSent.append(newWord)
            
    cleaned.append(newSent)

In [12]:
print(cleaned[0])
print(len(cleaned))

['RT', '@mayasolovely', ':', 'As', 'woman', 'complain', 'cleaning', 'house', '.', '&', 'man', 'always', 'take', 'trash', '...']
24783


In [13]:
maxWords = max(map(len, cleaned))

In [14]:
def create_seq(sent, vocab, maxWords):
    n = len(sent)
    numZeros = maxWords - n
    result = [0]*numZeros
    
    for word in sent:
        if word in vocab:
            result.append(vocab[word] + 2)
        else:
            result.append(1)
            
    return result

In [15]:
sequences = np.array(list(map(lambda x: create_seq(x, wordDict, maxWords), cleaned)))

In [24]:
c = lstm_cleaner()
sequences = c.clean_data(df['tweet'])

In [25]:
y_cat = np.zeros((len(y), 3))
for i, l in enumerate(y):
    y_cat[i, l] = 1

In [26]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(sequences, y_cat, test_size = .1)

In [27]:
print(max(wordDict.values()))

41243


In [30]:
model = hate_speech_model()
model.build_model()
model.fit(X_train, y_train, epochs = 100, class_weight = {0: 1, 1: .05, 2: .15})
preds = model.predict(X_test)

>> Compiled...
Train on 21188 samples, validate on 1116 samples
Epoch 1/100
21188/21188 [==============================] - 64s 3ms/sample - loss: 0.1324 - val_loss: 0.1424
Epoch 2/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1311 - val_loss: 0.1407
Epoch 3/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1293 - val_loss: 0.1382
Epoch 4/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1268 - val_loss: 0.1355
Epoch 5/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1253 - val_loss: 0.1346
Epoch 6/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1249 - val_loss: 0.1344
Epoch 7/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1245 - val_loss: 0.1341
Epoch 8/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1242 - val_loss: 0.1338
Epoch 9/100
21188/21188 [==============================] - 57s 3ms/sampl

In [31]:
print(metrics.roc_auc_score(y_test, preds))

0.8773220223226691


In [32]:
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(preds, axis = 1) 

In [33]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.20      0.68      0.31       154
           1       0.94      0.82      0.87      1921
           2       0.84      0.61      0.71       404

    accuracy                           0.77      2479
   macro avg       0.66      0.70      0.63      2479
weighted avg       0.88      0.77      0.81      2479



In [34]:
print(metrics.confusion_matrix(y_true, y_pred))

[[ 105   44    5]
 [ 313 1567   41]
 [ 102   57  245]]


In [29]:
import tensorflow as tf
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.keras.backend import set_session
tf.keras.backend.clear_session()  # For easy reset of notebook state.

config_proto = tf.ConfigProto()
off = rewriter_config_pb2.RewriterConfig.OFF
config_proto.graph_options.rewrite_options.arithmetic_optimization = off
session = tf.Session(config=config_proto)
set_session(session)